In [2]:
import pandas as pd
import numpy as np
import pymongo
import dns

## Read Files

In [3]:
predictions_file = 'predictions/rfc_bigtuesday_pruned.csv'

In [4]:
predictions = pd.read_csv(predictions_file)

In [5]:
predictions.head()

,% Donations Under $200,AFN120207,AGE135214,AGE295214,AGE775214,Age,BPS030214,BZA010213,BZA110213,BZA115213,...,raise_income_tax,ratify_NA_trade,rent_control,state,tax_wealthy_assets,used_in_model,vote_incarcerated,votes,winner,winner_state
0,0.3624,43204,4.3,18.3,21.6,77,99,567,2927,1.4,...,0.0,1.0,0.0,ID,0.0,0,0.0,653,1,1
1,0.3624,112044,7.7,27.6,14.8,77,361,2455,27449,8.0,...,0.0,1.0,0.0,ID,0.0,0,0.0,1505,1,1
2,0.3624,10730,7.8,27.8,8.6,77,28,418,1882,5.1,...,0.0,1.0,0.0,ID,0.0,0,0.0,613,0,1
3,0.3624,11068,5.2,19.7,21.7,77,7,355,4166,-0.4,...,0.0,1.0,0.0,ID,0.0,0,0.0,577,1,1
4,0.3624,2596,9.9,30.5,13.9,77,8,150,2027,-3.1,...,0.0,1.0,0.0,ID,0.0,0,0.0,166,1,1


In [35]:
predictions = predictions.sort_values(by=['fips'], ascending=False)

In [72]:
predictions[['last name', 'winner']]

,last name,winner
3229,Biden,-1
3228,Sanders,-1
3226,Sanders,-1
3227,Biden,-1
3225,Biden,-1
...,...,...
3293,Biden,1
4530,Sanders,0
3294,Biden,1
4531,Sanders,0


In [15]:
unique_fips = predictions['fips'].unique()
unique_states = predictions['state'].unique()

## Set up Connection to Mongo DB

In [14]:
username = 'Terra925'
password = 'H%40mmond271'
conn = 'mongodb+srv://' + username + ':' + password + '@cluster0-paegd.mongodb.net/test?retryWrites=true&w=majority'
client = pymongo.MongoClient(conn)
db = client['primary_predictions']
collection = db['county_results']

In [49]:
post = {
    'results': []
}


for fip in unique_fips:
    
    df = predictions[predictions['fips'] == fip]
    predicted_winner = df[df['predicted_winner'] == 1]['last name']
    actual_winner = df[df['winner'] == 1]['last name']
    try:
        predicted_winner = df[df['predicted_winner'] == 1]['last name']
        predicted_winner = predicted_winner.iloc[0]
    except:
        predicted_winner = 'Undetermined'
    try:
        predicted_loser = df[df['predicted_winner'] == 0]['last name']
        predicted_loser = predicted_loser.iloc[0]
    except:
        predicted_loser = 'Undetermined'
    try: 
        model = df[df['predicted_winner'] == -1]['predicted_winner'].iloc[0]
        used_in_model = 'Yes'
    except:
        used_in_model = 'No'
    try:
        actual_winner = df[df['winner'] == 1]['last name']
        actual_winner = actual_winner.iloc[0]
    except:
        actual_winner = 'Undetermined'
    try:
        actual_loser = df[df['winner'] == 0]['last name']
        actual_loser = actual_loser.iloc[0]
    except:
        actual_loser = 'Undetermined'
            
    obj = {
        'fip': int(fip),
        'state_abrv': str(df['state'].iloc[0]),
        'predicted_winner': str(predicted_winner),
        'predicted_loser': str(predicted_loser),
        'actual_winner': str(actual_winner),
        'actual_loser': str(actual_loser),
        'used_in_model': used_in_model
    }
    post['results'].append(obj)
id = collection.insert_one(post).inserted_id


In [65]:
predictions[predictions['fips'] == '6115']

c:\users\terra\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,% Donations Under $200,AFN120207,AGE135214,AGE295214,AGE775214,Age,BPS030214,BZA010213,BZA110213,BZA115213,...,raise_income_tax,ratify_NA_trade,rent_control,state,tax_wealthy_assets,used_in_model,vote_incarcerated,votes,winner,winner_state


In [84]:
collection = db['state_results']
post = {
    'results': []
}


for state in unique_states:
    
    df = predictions[predictions['state'] == state]
    sanders_predicted = df[df['last name'] == 'Sanders']['predicted_winner'].sum()
    biden_predicted = df[df['last name'] == 'Biden']['predicted_winner'].sum()
    sanders_actual = df[df['last name'] == 'Sanders']['winner'].sum()
    biden_actual = df[df['last name'] == 'Biden']['winner'].sum()
    
    if sanders_predicted < 0:
        used_in_model = 'Yes'
        state_predicted_winner = 'Undetermined'
        state_predicted_loser = 'Undetermined'
    elif sanders_predicted > biden_predicted:
        state_predicted_winner = 'Sanders'
        state_predicted_loser = 'Biden'
        used_in_model = 'No'
    else: 
        state_predicted_winner = 'Biden'
        state_predicted_loser = 'Sanders'
        used_in_model = 'No'
        
    if sanders_actual < 0:
        state_actual_winner = 'Undetermined'
        state_actual_loser = 'Undetermined'
    elif sanders_actual > biden_actual:
        state_actual_winner = 'Sanders'
        state_actual_loser = 'Biden'
    elif biden_actual > sanders_actual:
        state_actual_winner = 'Biden'
        state_actual_loser = 'Sanders'
    else:
        state_actual_winner = 'Undetermined'
        state_actual_loser = 'Undetermined'
        
    
    obj = {
        'state': state,
        'used_in_model': used_in_model,
        'predicted_winner': state_predicted_winner,
        'predicted_loser': state_predicted_loser,
        'actual_winner': state_actual_winner,
        'actual_loser': state_actual_loser
    }
    post['results'].append(obj)


id = collection.insert_one(post).inserted_id

In [83]:
post

{'results': [{'state': 'ID',
   'used_in_model': 'No',
   'predicted_winner': 'Biden',
   'predicted_loser': 'Sanders',
   'actual_winner': 'Biden',
   'actual_loser': 'Sanders'},
  {'state': 'MI',
   'used_in_model': 'No',
   'predicted_winner': 'Biden',
   'predicted_loser': 'Sanders',
   'actual_winner': 'Biden',
   'actual_loser': 'Sanders'},
  {'state': 'MS',
   'used_in_model': 'No',
   'predicted_winner': 'Biden',
   'predicted_loser': 'Sanders',
   'actual_winner': 'Biden',
   'actual_loser': 'Sanders'},
  {'state': 'MO',
   'used_in_model': 'No',
   'predicted_winner': 'Biden',
   'predicted_loser': 'Sanders',
   'actual_winner': 'Biden',
   'actual_loser': 'Sanders'},
  {'state': 'WA',
   'used_in_model': 'No',
   'predicted_winner': 'Sanders',
   'predicted_loser': 'Biden',
   'actual_winner': 'Biden',
   'actual_loser': 'Sanders'},
  {'state': 'ND',
   'used_in_model': 'No',
   'predicted_winner': 'Sanders',
   'predicted_loser': 'Biden',
   'actual_winner': 'Sanders',
   '